In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import requests


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


data/accidents-in-france-from-2005-to-2016/users.csv
data/accidents-in-france-from-2005-to-2016/places.csv
data/accidents-in-france-from-2005-to-2016/holidays.csv
data/accidents-in-france-from-2005-to-2016/caracteristics.csv
data/accidents-in-france-from-2005-to-2016/vehicles.csv


In [2]:
caracteristics_path = 'data/accidents-in-france-from-2005-to-2016/caracteristics.csv'
caracteristics = pd.read_csv(caracteristics_path, encoding = 'latin-1')

/opt/miniconda3/envs/scapper/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def invalide_rows(caracteristics):
    
    
    adr_na = caracteristics['adr'].isna()
    
    lat_na = caracteristics['lat'].isna() | caracteristics['lat'].isnull()
    long_na = caracteristics['long'].isna() | caracteristics['long'].isnull()
    
    
    lat_zero = caracteristics['lat'] == 0
    long_zero = (caracteristics['long'] == 0) | (caracteristics['long'] == '0')
    
    invalid_adr =  adr_na & (lat_zero | long_zero | lat_na | long_na)
    
    return invalid_adr

In [31]:
invalid_rows = invalide_rows(caracteristics)
valid_rows = ~invalid_rows

In [119]:
valide_caracteristics = caracteristics[valid_rows]
# garder tous ce qui sont dans la metropole
in_metropole = (valide_caracteristics['gps'] == 'M') | valide_caracteristics['gps'].isna() & (valide_caracteristics['dep'] < 951)
valide_caracteristics = valide_caracteristics[in_metropole]

In [86]:
def adr_could_be_filled(valide_caracteristics):
    
    adr_not_na = valide_caracteristics['adr'].notna()
    
    lat_na = valide_caracteristics['lat'].isna() | valide_caracteristics['lat'].isnull()
    long_na = valide_caracteristics['long'].isna() | valide_caracteristics['long'].isnull()
    
    code_com_dep_na  = valide_caracteristics['com'].isna() | valide_caracteristics['com'].isnull() 
    code_com_dep_na |= valide_caracteristics['dep'].isna() | valide_caracteristics['dep'].isnull()

    
    lat_zero = valide_caracteristics['lat'] == 0
    long_zero = (valide_caracteristics['long'] == 0) | (valide_caracteristics['long'] == '0')
    
    return adr_not_na & (~code_com_dep_na) & (lat_na | long_na | lat_zero | long_zero)
    

In [120]:
valide_adr = adr_could_be_filled(valide_caracteristics)

valide_adr_caracteristics = valide_caracteristics[valide_adr]

In [34]:
def formate_com(com):
    com = str(int(com))
    extra_zero = '0' * (3 - len(com))
    return extra_zero + com 


In [121]:
com_new = valide_adr_caracteristics['com'].map(lambda x: formate_com(x))
dep_new = valide_adr_caracteristics['dep'].map(lambda x: str(x)[:2])

valide_adr_caracteristics['dep_com'] = dep_new + com_new

<ipython-input-121-85aa8651ef1a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valide_adr_caracteristics['dep_com'] = dep_new + com_new


In [89]:
import time
import requests
import threading
import numpy as np
import concurrent.futures

In [122]:
lock = threading.Lock()

def get_columns():
    return [
        ('city', ''), ('name', ''), ('score', float(0.0)), 
        ('postcode', -1), ('longitude', float(0.0)), ('latitude', float(0.0))
    ]

def add_columns(df):
    for col in get_columns():
        df[col[0]] = col[1]
    return df


start = time.time()
valide_adr_caracteristics = valide_adr_caracteristics.iloc[:10000]
valide_adr_caracteristics = add_columns(valide_adr_caracteristics)

url = 'http://api-adresse.data.gouv.fr/search/?q={}'
adr_dep_com = (valide_adr_caracteristics['adr'] + ' ') + valide_adr_caracteristics['dep_com']
valide_adr_caracteristics['urls'] = adr_dep_com.map(url.format)


def lat_long_from_adr(index_carac):
    
    global lock
    i, carac = index_carac # valide_adr_caracteristics.iloc[i]
        
    response = requests.get(carac['urls'])
    resp_json = response.json()
    if len(resp_json['features']) < 1: return
    feature = resp_json['features'][0]
    
    global valide_adr_caracteristics
    with lock:
        valide_adr_caracteristics.loc[i, 'city'] = feature['properties']['city']
        valide_adr_caracteristics.loc[i, 'name'] = feature['properties']['name']
        valide_adr_caracteristics.loc[i, 'score'] = round(feature['properties']['score'], 3)
        valide_adr_caracteristics.loc[i, 'postcode'] = feature['properties']['postcode']
        valide_adr_caracteristics.loc[i, 'longitude'] = feature['geometry']['coordinates'][0]
        valide_adr_caracteristics.loc[i, 'latitude'] = feature['geometry']['coordinates'][1]

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(lat_long_from_adr, valide_adr_caracteristics.iterrows())


print('end', time.time() - start, 's')
cols = np.array(get_columns())[:, 0]
print(cols)
valide_adr_caracteristics.loc[:, cols].to_csv('adr_complet.csv', index_label='Index', index=True)

end 423.1078579425812 s
['city' 'name' 'score' 'postcode' 'longitude' 'latitude']


In [123]:
invalide = valide_adr_caracteristics[valide_adr_caracteristics['postcode'] == -1]['postcode'].count() 
total = valide_adr_caracteristics['postcode'].count()

print(invalide)
print(total)
'loss {} %'.format(invalide / total)

458
10000


'loss 0.0458 %'

In [ ]:
#unsent_requests = (grequests.get(u, session=requests.Session(), stream=False) for u in urls)
#valide_adr_caracteristics = add_columns(valide_adr_caracteristics)
#responses = grequests.map(unsent_requests, size=10)
#for i, response in enumerate(responses):
#    lat_long_from_adr(response, valide_adr_caracteristics, i)

In [124]:
for i, _ in valide_adr_caracteristics.iterrows():
    pass
#i['longitude'] = 1.0
#valide_adr_caracteristics.loc[i, 'city'] = 'City'
print(i)
valide_adr_caracteristics.loc[i]

43589


Num_Acc                                           201600043590
an                                                          16
mois                                                        12
jour                                                        11
hrmn                                                      2305
lum                                                          5
agg                                                          2
int                                                          2
atm                                                          1
col                                                          3
com                                                         68
adr                                        CORNEILLES (BD DES)
gps                                                        NaN
lat                                                        NaN
long                                                       NaN
dep                                                    